In [ ]:
import os
import sys
# 把父路径添加到检索目标中
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from config.config import llm_path
######################################################
from swift.llm import DatasetName, ModelType, SftArguments, sft_main, InferArguments, merge_lora_main, infer_main

In [ ]:
# 设置需要使用的显卡设备号
# nvidia-smi 查看显卡编号
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4'

In [ ]:
sft_args = SftArguments(
    model_cache_dir=llm_path,
    model_type=ModelType.qwen_72b_chat,
    dataset='ms-bench',
    train_dataset_sample=10000,
    eval_steps=20,
    logging_steps=5,
    output_dir="output",
    lora_target_modules="ALL",
    self_cognition_sample=500,
    model_name=["小光", "Xiao Guang"],
    model_author=["小光团队", "Team of Xiao Guang"]
)
output = sft_main(sft_args)
best_model_checkpoint = output["best_model_checkpoint"]

In [ ]:
infer_args = InferArguments(
    model_cache_dir=llm_path,
    # infer_backend="pt", 
    # 设置需要使用的显卡数量，需要与环境变量 CUDA_VISIBLE_DEVICES 配合
    # 与 swift deploy 不同，显卡数可以不是偶数
    tensor_parallel_size=5, 
    # gpu_memory_utilization=0.72, 
    # 若使用了 checkpoint 则无需定义 model type
    ckpt_dir=best_model_checkpoint,
    eval_human=True,
    merge_lora_and_save=True
)

# 开始推理测试（ 并执行微调合并 ）
result = infer_main(infer_args)